## Imports

In [ ]:
import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image

from tqdm import tqdm

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

## Model Settings

In [ ]:
RANDOM_SEED = 1
LEARNING_RATE = 0.0001
BATCH_SIZE = 32
NUM_EPOCHS = 20

RESOLUTION = 200

NUM_FEATURES = RESOLUTION*RESOLUTION
NUM_CLASSES = 66

DEVICE = "cuda:0"
GRAYSCALE = False

In [ ]:
transform = transforms.Compose([
    transforms.Resize(RESOLUTION),
    transforms.CenterCrop(RESOLUTION),
    transforms.ToTensor()
])

path = 'CV_data/Extracted_dataset_input_combined'
dataset = datasets.ImageFolder(root=path, transform=transform)


# Get class to index mapping
class_to_idx = dataset.class_to_idx

# Define index to class mapping
idx_to_class = {v: k for k, v in class_to_idx.items()}

train_size = int(0.6 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

# Split the dataset into training set and test set
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size, test_size])


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=True)

val_loader = DataLoader(dataset=val_dataset, 
                        batch_size=BATCH_SIZE, 
                        shuffle=False)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=BATCH_SIZE, 
                         shuffle=True)

# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    print('Labels:', labels)
    break


In [ ]:
device = torch.device(DEVICE)
torch.manual_seed(0)

for epoch in range(2):

    for batch_idx, (x, y) in enumerate(train_loader):
        
        print('Epoch:', epoch+1, end='')
        print(' | Batch index:', batch_idx, end='')
        print(' | Batch size:', y.size()[0])
        
        x = x.to(device)
        y = y.to(device)
        break

# Model

In [ ]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out




class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes, grayscale):
        self.inplanes = 64
        if grayscale:
            in_dim = 1
        else:
            in_dim = 3
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(in_dim, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, (2. / n)**.5)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        
        x = x.view(x.size(0), -1)
        logits = self.fc(x)
        probas = F.softmax(logits, dim=1)
        return logits, probas



def resnet50(num_classes):
    """Constructs a ResNet-50 model."""
    model = ResNet(block=Bottleneck, 
                   layers=[3, 4, 6, 3],
                   num_classes=NUM_CLASSES,
                   grayscale=GRAYSCALE)
    return model


In [ ]:
torch.manual_seed(RANDOM_SEED)

model = resnet50(NUM_CLASSES)
model.to(DEVICE)
 
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  

In [ ]:
def compute_accuracy(model, data_loader, device, mode="train"):
    correct_pred, num_examples = 0, 0
    for batch_idx, (features, targets) in tqdm(enumerate(data_loader)):
            
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)

        logits, probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
        
        if not batch_idx % 50:
            accuracy = correct_pred.float() / num_examples * 100
            with open(f'CV_data/models/{model_name}-{LEARNING_RATE}-{mode}-accuracy.csv', 'a') as fp:
                fp.write(f"{epoch+1}, {batch_idx}, {accuracy}\n")
            correct_pred, num_examples = 0, 0



In [ ]:
def compute_loss(model, data_loader, class_weights, device, mode="val"):
    losses = []
    dataset_len = len(data_loader)
    for batch_idx, (features, targets) in tqdm(enumerate(data_loader)):
        
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)
        class_weights = class_weights.to(DEVICE)
            
        logits, _ = model(features)
        loss = F.cross_entropy(logits, targets)
        losses.append(loss)
        
        if not batch_idx % 50:
            mean_loss = sum(losses) / len(losses)
            with open(f'CV_data/models/{model_name}-{LEARNING_RATE}-{mode}-loss.csv', 'a') as fp:
                fp.write(f"{epoch+1}, {batch_idx}, {mean_loss}\n")
            losses.clear()

In [ ]:
model_name = "00-model"

## Training

In [11]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    
    model.train()
    losses = []
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)
        class_weights = class_weights.to(DEVICE)
            
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        loss = F.cross_entropy(logits, targets)
        losses.append(loss)

        optimizer.zero_grad()
        
        loss.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %04d/%04d | Cost: %.4f' 
                   %(epoch+1, NUM_EPOCHS, batch_idx,
                     len(train_loader), loss))
    
            mean_loss = sum(losses) / len(losses)
            with open(f'CV_data/models/{model_name}-{LEARNING_RATE}-train-loss.csv', 'a') as fp:
                fp.write(f"{epoch+1}, {batch_idx}, {mean_loss}\n")
            losses.clear()


    model.eval()
    with torch.set_grad_enabled(False): # save memory during inference
        compute_accuracy(model, train_loader, device=DEVICE, mode="train")
        compute_loss(model, val_loader, class_weights, device=DEVICE, mode="val")
        compute_accuracy(model, val_loader, device=DEVICE, mode="val")
    
    file_path = f'CV_data/models/{model_name}-{LEARNING_RATE}-epoch{epoch+1}.pth'
    torch.save(model.state_dict(), file_path)
        
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
    
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

Epoch: 001/020 | Batch 0000/1284 | Cost: 4.2649
Epoch: 001/020 | Batch 0050/1284 | Cost: 3.8272
Epoch: 001/020 | Batch 0100/1284 | Cost: 3.5944
Epoch: 001/020 | Batch 0150/1284 | Cost: 3.7192
Epoch: 001/020 | Batch 0200/1284 | Cost: 3.3314


/data/leuven/354/vsc35486/miniconda3/lib/python3.10/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch: 001/020 | Batch 0250/1284 | Cost: 3.0997
Epoch: 001/020 | Batch 0300/1284 | Cost: 3.1404
Epoch: 001/020 | Batch 0350/1284 | Cost: 2.5295
Epoch: 001/020 | Batch 0400/1284 | Cost: 2.7810
Epoch: 001/020 | Batch 0450/1284 | Cost: 2.8899
Epoch: 001/020 | Batch 0500/1284 | Cost: 2.6300
Epoch: 001/020 | Batch 0550/1284 | Cost: 2.3803
Epoch: 001/020 | Batch 0600/1284 | Cost: 2.2133
Epoch: 001/020 | Batch 0650/1284 | Cost: 2.6430
Epoch: 001/020 | Batch 0700/1284 | Cost: 1.9110
Epoch: 001/020 | Batch 0750/1284 | Cost: 1.8039
Epoch: 001/020 | Batch 0800/1284 | Cost: 2.4884
Epoch: 001/020 | Batch 0850/1284 | Cost: 1.8090
Epoch: 001/020 | Batch 0900/1284 | Cost: 1.6273
Epoch: 001/020 | Batch 0950/1284 | Cost: 1.8678
Epoch: 001/020 | Batch 1000/1284 | Cost: 1.6614
Epoch: 001/020 | Batch 1050/1284 | Cost: 1.4066
Epoch: 001/020 | Batch 1100/1284 | Cost: 1.3027
Epoch: 001/020 | Batch 1150/1284 | Cost: 1.4242
Epoch: 001/020 | Batch 1200/1284 | Cost: 1.4810
Epoch: 001/020 | Batch 1250/1284 | Cost:

1284it [02:12,  9.72it/s]
428it [00:55,  7.74it/s]
428it [00:40, 10.50it/s]


Time elapsed: 8.59 min
Epoch: 002/020 | Batch 0000/1284 | Cost: 1.7066
Epoch: 002/020 | Batch 0050/1284 | Cost: 1.1887
Epoch: 002/020 | Batch 0100/1284 | Cost: 1.5444
Epoch: 002/020 | Batch 0150/1284 | Cost: 1.0590
Epoch: 002/020 | Batch 0200/1284 | Cost: 1.4377
Epoch: 002/020 | Batch 0250/1284 | Cost: 1.1466
Epoch: 002/020 | Batch 0300/1284 | Cost: 0.7916
Epoch: 002/020 | Batch 0350/1284 | Cost: 1.1159
Epoch: 002/020 | Batch 0400/1284 | Cost: 1.1256
Epoch: 002/020 | Batch 0450/1284 | Cost: 0.9754
Epoch: 002/020 | Batch 0500/1284 | Cost: 1.2763
Epoch: 002/020 | Batch 0550/1284 | Cost: 0.6040
Epoch: 002/020 | Batch 0600/1284 | Cost: 0.7611
Epoch: 002/020 | Batch 0650/1284 | Cost: 1.0281
Epoch: 002/020 | Batch 0700/1284 | Cost: 0.5661
Epoch: 002/020 | Batch 0750/1284 | Cost: 1.0329
Epoch: 002/020 | Batch 0800/1284 | Cost: 0.8230
Epoch: 002/020 | Batch 0850/1284 | Cost: 0.5291
Epoch: 002/020 | Batch 0900/1284 | Cost: 0.4634
Epoch: 002/020 | Batch 0950/1284 | Cost: 0.5387
Epoch: 002/020 | 

...

Time elapsed: 133.31 min
Epoch: 019/020 | Batch 0000/1284 | Cost: 0.0181
Epoch: 019/020 | Batch 0050/1284 | Cost: 0.0015
Epoch: 019/020 | Batch 0100/1284 | Cost: 0.0021
Epoch: 019/020 | Batch 0150/1284 | Cost: 0.0456
Epoch: 019/020 | Batch 0200/1284 | Cost: 0.0706
Epoch: 019/020 | Batch 0250/1284 | Cost: 0.0017
Epoch: 019/020 | Batch 0300/1284 | Cost: 0.0756
Epoch: 019/020 | Batch 0350/1284 | Cost: 0.1165
Epoch: 019/020 | Batch 0400/1284 | Cost: 0.0068
Epoch: 019/020 | Batch 0450/1284 | Cost: 0.0769
Epoch: 019/020 | Batch 0500/1284 | Cost: 0.0184
Epoch: 019/020 | Batch 0550/1284 | Cost: 0.1105
Epoch: 019/020 | Batch 0600/1284 | Cost: 0.0662
Epoch: 019/020 | Batch 0650/1284 | Cost: 0.0307
Epoch: 019/020 | Batch 0700/1284 | Cost: 0.0350
Epoch: 019/020 | Batch 0750/1284 | Cost: 0.0009
Epoch: 019/020 | Batch 0800/1284 | Cost: 0.0054
Epoch: 019/020 | Batch 0850/1284 | Cost: 0.0028
Epoch: 019/020 | Batch 0900/1284 | Cost: 0.0037
Epoch: 019/020 | Batch 0950/1284 | Cost: 0.0382
Epoch: 019/020 

1284it [01:56, 11.04it/s]
428it [00:36, 11.65it/s]
428it [00:36, 11.72it/s]


Time elapsed: 140.37 min
Epoch: 020/020 | Batch 0000/1284 | Cost: 0.0065
Epoch: 020/020 | Batch 0050/1284 | Cost: 0.0028
Epoch: 020/020 | Batch 0100/1284 | Cost: 0.0031
Epoch: 020/020 | Batch 0150/1284 | Cost: 0.0479
Epoch: 020/020 | Batch 0200/1284 | Cost: 0.0417
Epoch: 020/020 | Batch 0250/1284 | Cost: 0.0278
Epoch: 020/020 | Batch 0300/1284 | Cost: 0.0433
Epoch: 020/020 | Batch 0350/1284 | Cost: 0.0093
Epoch: 020/020 | Batch 0400/1284 | Cost: 0.1345
Epoch: 020/020 | Batch 0450/1284 | Cost: 0.1095
Epoch: 020/020 | Batch 0500/1284 | Cost: 0.1126
Epoch: 020/020 | Batch 0550/1284 | Cost: 0.0044
Epoch: 020/020 | Batch 0600/1284 | Cost: 0.2095
Epoch: 020/020 | Batch 0650/1284 | Cost: 0.0294
Epoch: 020/020 | Batch 0700/1284 | Cost: 0.0087
Epoch: 020/020 | Batch 0750/1284 | Cost: 0.1749
Epoch: 020/020 | Batch 0800/1284 | Cost: 0.3778
Epoch: 020/020 | Batch 0850/1284 | Cost: 0.0345
Epoch: 020/020 | Batch 0900/1284 | Cost: 0.0151
Epoch: 020/020 | Batch 0950/1284 | Cost: 0.0137
Epoch: 020/020 

1284it [01:55, 11.07it/s]
428it [00:36, 11.77it/s]
428it [00:36, 11.79it/s]


Time elapsed: 147.40 min
Total Training Time: 147.40 min


## Evaluation

In [13]:
DEVICE = "cuda:0"
device = torch.device(DEVICE)

torch.manual_seed(RANDOM_SEED)

model_weights = torch.load(f"CV_data/models/{model_name}-{LEARNING_RATE}-epoch20.pth", map_location=torch.device('cpu'))

model = resnet50(NUM_CLASSES) 

# Loading the weights to the model
model.load_state_dict(model_weights)

model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

# Confusion matrix

!pip install seaborn

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd

y_pred = []
y_true = []

model.eval()
with torch.set_grad_enabled(False):
    for inputs, labels in test_loader:
            inputs = inputs.to(DEVICE)
            _, classes = model(inputs) # Feed Network

            output = (torch.max(torch.exp(classes), 1)[1]).data.cpu().numpy()
            y_pred.extend(output) # Save Prediction

            labels = labels.data.cpu().numpy()
            y_true.extend(labels) # Save Truth

In [ ]:
# Build confusion matrix
cf_matrix = confusion_matrix(y_true, y_pred)
classes = dataset.class_to_idx.keys()
df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [i for i in classes],
                     columns = [i for i in classes])
plt.figure(figsize=(30, 25))
sn.heatmap(df_cm, annot=True)
plt.savefig(f'CV_data/conf_matrix/{model_name}-output.pdf')

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd

y_pred = []
y_true = []

model.eval()
with torch.set_grad_enabled(False):
    for inputs, labels in test_loader:
            inputs = inputs.to(DEVICE)
            _, classes = model(inputs) # Feed Network

            output = (torch.max(torch.exp(classes), 1)[1]).data.cpu().numpy()
            y_pred.extend(output) # Save Prediction

            labels = labels.data.cpu().numpy()
            y_true.extend(labels) # Save Truth



In [ ]:
# Build confusion matrix
cf_matrix = confusion_matrix(y_true, y_pred)
classes = dataset.class_to_idx.keys()
df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [i for i in classes],
                     columns = [i for i in classes])
plt.figure(figsize=(30, 25))
sn.heatmap(df_cm, annot=True)
plt.savefig(f'CV_data/conf_matrix/{model_name}-output.pdf')

# Test examples

In [ ]:
for batch_idx, (features, targets) in enumerate(test_loader):

    features = features
    targets = targets
    break

NUM_IMG = 4
fig, axs = plt.subplots(1, NUM_IMG)
imgs = []
classes = []

for idx in range(NUM_IMG):
    nhwc_img = np.transpose(features[idx], axes=(1, 2, 0))
    imgs.append(nhwc_img)

    model.eval()
    _ , probas = model(features.to(device)[idx, None])
    probas = probas.cpu().detach().numpy()

    classes.append(
        {"class": idx_to_class[np.argmax(probas[0])],
         "proba": np.max(probas[0])})

for i, ax in enumerate(axs.flat):
    if i < len(imgs):
        ax.imshow(imgs[i])
        ax.set_title(f'Class: {classes[i]["class"]}, \nProbability:  {classes[i]["proba"]:.4f}', fontsize=8)
    ax.set_xticks([])
    ax.set_yticks([])


# Own photos

In [ ]:
import os
path = "CV_data/own_photos/"
classes = os.listdir(path)
classes = [fldr for fldr in classes if not fldr.startswith('.')]

print(classes)

transform = transforms.Compose([
    transforms.Resize(RESOLUTION),
    transforms.CenterCrop(RESOLUTION),
    transforms.ToTensor()
])

custom_test_dataset = datasets.ImageFolder(root=path, transform=transform)

custom_test_loader = DataLoader(dataset=custom_test_dataset, 
                         batch_size=BATCH_SIZE, 
                         shuffle=True)


In [ ]:
for batch_idx, (features, targets) in enumerate(custom_test_loader):

    features = features
    targets = targets
    break

NUM_IMG = 4
fig, axs = plt.subplots(1, NUM_IMG)
imgs = []
classes = []

for idx in range(NUM_IMG):
    nhwc_img = np.transpose(features[idx], axes=(1, 2, 0))
    imgs.append(nhwc_img)

    model.eval()
    _ , probas = model(features.to(device)[idx, None])
    probas = probas.cpu().detach().numpy()

    classes.append(
        {"class": idx_to_class[np.argmax(probas[0])],
         "proba": np.max(probas[0])})

for i, ax in enumerate(axs.flat):
    if i < len(imgs):
        ax.imshow(imgs[i])
        ax.set_title(f'Class: {classes[i]["class"]}, \nProbability:  {classes[i]["proba"]:.4f}', fontsize=8)
    ax.set_xticks([])
    ax.set_yticks([])

In [ ]:
from lib.nutrifacts import retrieve_nutrition_facts
import json

from IPython.core.display import HTML
HTML("&micro;")

print(json.dumps(retrieve_nutrition_facts(classes[2]["class"]), indent=4))